In [ ]:
# coming up with a moodel which will predict the next characters in the play
# input = sequence of characters , part of play
# output = predict next set of characters, will predict one character at a tiime


In [1]:
from keras.preprocessing import  sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [8]:
# dataset, download the file for romeo juliet play
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [7]:
# load your own data


# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [12]:
# read the file contents

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('length of char in the file is ',len(text))
print(text[:250])


# may our RNN predict the breakline and also the spaces 

length of char in the file is  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [22]:
# Encoding
# cuz the data isnt encoded to numbers we have to do it ourselves, this time we assign each character a number
# as there are finite number of characters, alphabets spaces colon and breakline where are infinite number of combo of charcs, ie, words

# sorted unique set of characters from text are taken
vocab = sorted(set(text)) 

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)


print('char2idx is ',char2idx)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

char2idx is  {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [19]:
print(text[:250])
print('encoded ',text_as_int[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

encoded  [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 

In [24]:
# we convert our numeric index to text

def int_to_text(integers):
  try:
    integers = integers.numpy()
  except:
    pass
  return ''.join(idx2char[integers])

print(int_to_text(text_to_int(text[:13])))

First Citizen


In [25]:
# creating training example

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# create char dataset
char_dataset =  tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
# using batch method to convert this stream of characters into  batches of desired length

In [26]:
sequence =  char_dataset.batch( seq_length+1, drop_remainder=True)


In [28]:
# shifting the same words left by on char = input : hell
# shifting the same words right by on char = output: ello
# hell & ello ---> 'o' is predicted and the hell+o will be hello
def split_input_data(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text # hell ello

dataset = sequence.map(split_input_data)

In [32]:
for x,y in dataset.take(2):
  print('\nEXAMPLE')
  print('\nINPUT')
  print(int_to_text(x))
  print('\nOUTPUT')
  print(int_to_text(y))


EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 

EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [34]:
# making training batches

BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 64
RNN_UNITS = 1024


# Buffer is to shuffle the data around

BUFFER_SIZE = 10000 


data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [39]:
# building the model

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                                         batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequence=True,
                                                    stateful=True,
                                                    recurrent_initializer='glorot_uniform'
                                                    ),
                               tf.keras.Dense(vocab_size)
  ])

  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

TypeError: ignored